In [1]:
import numpy as np
X = {}
y = {}
X["train"] = np.load("./train_feature_save.npy",allow_pickle=True )
y["train"] = np.load("./train_label_save.npy")
X["dev"] = np.load("./test_feature_save.npy")
y["dev"] = np.load("./test_label_save.npy")
X["train"].shape, X["dev"].shape

((6920, 768), (872, 768))

In [2]:
y["train"]=y["train"].squeeze()
y["dev"]=y["dev"].squeeze()
y["train"].shape, y["dev"].shape

((6920,), (872,))

In [5]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=0.1)
model.fit(X["train"], y["train"])
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
model.score(X["dev"], y["dev"])

/opt/anaconda3/envs/torch/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.819954128440367

In [6]:
import sklearn.metrics as mc
predl = model.predict(X["dev"])
mc.f1_score(y["dev"], predl)

0.8302702702702703

In [7]:
fpr, tpr, thresholds = mc.roc_curve(y["dev"], pred)
mc.auc(fpr, tpr)

0.9106887261092868

In [13]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
#g = np.sum(gradient_train, axis=0) / 6920
gradient_train.shape, gradient_dev.shape

((6920, 769), (872, 769))

In [14]:
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0]
np.all(np.linalg.eigvals(hessian) > 0)

True

In [15]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + 10 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [16]:
np.all(np.linalg.eigvals(hessian) > 0)

True

In [17]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(872, 6920)

In [18]:
np.save("old_prediction.npy", pred)

In [19]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > 0.5:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]

    X_k = np.delete(X["train"], top_k_index, axis=0)
    y_k = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    
    #print("prediction", prediction)
    

    return X_k, y_k, prediction

In [41]:
def new_train(k, dev_index, scores):
    X_k, y_k, prediction = Remove(k, scores, dev_index)
    
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None, None, None

    # Fit the model again
    model_k = LogisticRegression(penalty='l2', C=0.1)
    model_k.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    
    new = model_k.predict_proba(test_point)[0][1]
    change = -(model.predict_proba(test_point)[0][1] - new)
    #change = model_k.predict_proba(test_point)[0][1]-model.predict_proba(test_point)[0][1]
    flip = (model.predict(test_point) == model_k.predict(test_point))
    
    """
    print("change    ", change)
    print("old       ", model.predict_proba(test_point)[0][1])
    print()
    """
    
  
    return change, flip, prediction,new

# exact_k

In [42]:
def exact_k(test_idx):
    for k in range(1, np.sum(y["train"])):
        _, flip,_ = new_train(k, test_idx, delta_pred)
        if flip == False:
            return k
    return None

In [43]:
"""
# Loop over all dev points:
import warnings
warnings.filterwarnings("ignore")

exact_ks = []
for test_idx in range(X["dev"].shape[0]):
    k =exact_k(test_idx)
    print(test_idx, k)
    exact_ks.append(k)

exact_ks=np.array(exact_ks)
np.save("exact_ks_IP.npy", exact_ks)
"""

'\n# Loop over all dev points:\nimport warnings\nwarnings.filterwarnings("ignore")\n\nexact_ks = []\nfor test_idx in range(X["dev"].shape[0]):\n    k =exact_k(test_idx)\n    print(test_idx, k)\n    exact_ks.append(k)\n\nexact_ks=np.array(exact_ks)\nnp.save("exact_ks_IP.npy", exact_ks)\n'

# Find approximate k by IF

In [44]:
def pred_change(k, scores, test_idx):
    X_k, y_k, prediction = Remove(k, delta_pred, test_idx)
    

In [45]:
def approximate_k(test_idx):
    old = pred[test_idx].item()
    
    if pred[test_idx] > 0.5:
        top_k_index = np.flip(delta_pred[test_idx].argsort())
    else:
        top_k_index = delta_pred[test_idx].argsort()
    
    for k in range(1, y["train"].shape[0]):
        change = -np.sum(delta_pred[test_idx][top_k_index[:k]])
        #print("prediction", change)
        
        if old > 0.5 and old + change < 0.5:
            return k
        elif old < 0.5 and old + change > 0.5:
            return k
        
    return None

In [46]:
# Loop over all dev points:
import warnings
warnings.filterwarnings("ignore")

appro_ks = []
new_predictions = []
for test_idx in range(X["dev"].shape[0]):
    appro_k = approximate_k(test_idx)
    if appro_k != None:
        _, _, _, new_prediction = new_train(appro_k, test_idx, delta_pred)
        print(test_idx, appro_k)
        print("old", pred[test_idx], "new", new_prediction)
        print()
        appro_ks.append(appro_k)
        new_predictions.append(new_prediction)
    else:
        appro_ks.append(None)
        new_predictions.append(None)



1 89
old [0.33663131] new 0.7086901103306189

3 342
old [0.82612505] new 0.0215972261089372

4 528
old [0.85928724] new 0.0025358351871436692

7 23
old [0.55550462] new 0.6387963588963607

9 8
old [0.47227723] new 0.6138639411183603

10 3520
old [0.9088971] new 4.455030904885552e-11

13 1524
old [0.89142157] new 4.0486617082754935e-06

14 13
old [0.56304596] new 0.5337172242934316

15 84
old [0.642106] new 0.5570864978695117

16 162
old [0.72393983] new 0.3158110005789283

17 454
old [0.79955231] new 0.062210470380534434

20 7
old [0.5176194] new 0.42566024143901854

22 161
old [0.72510113] new 0.25254163907946975

23 891
old [0.86431778] new 0.001071858430975208

26 359
old [0.77267291] new 0.0822523266438322

27 156
old [0.72944327] new 0.3550606798619924

28 692
old [0.82491015] new 0.03606410350065733

29 24
old [0.62560836] new 0.5641097049312036

31 1461
old [0.91623555] new 9.710417572821027e-08

32 103
old [0.33541233] new 0.6905355019302346

34 175
old [0.72296014] new 0.20602

246 137
old [0.7064643] new 0.35849491981568604

247 153
old [0.29351422] new 0.6353024263972099

249 1962
old [0.88205757] new 1.6382196111290896e-05

250 2126
old [0.89722782] new 6.435488208807888e-07

253 17
old [0.44453494] new 0.36094401799766457

255 1293
old [0.88075561] new 1.9035397451941316e-05

256 214
old [0.21528532] new 0.9065489024077582

257 16
old [0.54136297] new 0.4987928648360893

258 179
old [0.75177377] new 0.11398676491978167

261 2370
old [0.90058967] new 1.832020464262966e-07

263 246
old [0.78708817] new 0.03561930462489753

264 380
old [0.78465424] new 0.11162088948441104

265 207
old [0.79166271] new 0.06638247692119147

266 79
old [0.69203497] new 0.08022594805237844

267 1463
old [0.89047895] new 3.2977630263288363e-06

270 3083
old [0.90282626] new 1.6748391022161801e-09

274 51
old [0.64491134] new 0.4197995773189604

276 311
old [0.81999967] new 0.08732740819143567

277 965
old [0.11004091] new 0.9999728036318307

279 3
old [0.50915737] new 0.582180587

477 85
old [0.29787891] new 0.5994116771332544

479 10
old [0.47101822] new 0.3884728046407793

480 46
old [0.60833557] new 0.5365354404051279

481 72
old [0.38550526] new 0.6375745819223718

482 599
old [0.15547396] new 0.9959240507787216

483 23
old [0.55655533] new 0.35574909691626694

484 916
old [0.13535825] new 0.9997519251997509

486 35
old [0.36249542] new 0.4223413467904747

487 21
old [0.42860111] new 0.48014180315592825

488 551
old [0.13948213] new 0.9982065586399562

489 47
old [0.38092724] new 0.5352918242209669

490 324
old [0.19861379] new 0.9246977948648167

492 334
old [0.77479752] new 0.03698576730407464

493 181
old [0.25992166] new 0.8299971164734973

494 557
old [0.17595225] new 0.9872197556315233

495 37
old [0.69967543] new 0.40092949719076426

496 35
old [0.58992685] new 0.3203667695039755

497 142
old [0.27793788] new 0.6836744087311044

498 1913
old [0.11648029] new 0.9999962774100929

499 3219
old [0.08612149] new 0.9999999999486413

500 787
old [0.14151226]

666 33
old [0.59934313] new 0.29550559697085443

667 65
old [0.64011539] new 0.4612400964226346

668 424
old [0.14890232] new 0.9698215284207029

669 495
old [0.1732954] new 0.9928275121305162

670 1557
old [0.12316851] new 0.9999953731793993

671 533
old [0.1321624] new 0.9984557270148631

672 269
old [0.20875201] new 0.9590647975156914

673 700
old [0.16006692] new 0.996128667917353

674 438
old [0.19799873] new 0.9811046056548496

675 111
old [0.34818149] new 0.7088114187222705

676 273
old [0.79151424] new 0.02244994706889538

677 201
old [0.74490503] new 0.2221371495623847

678 1162
old [0.87304655] new 0.00029690027952295725

680 403
old [0.80846898] new 0.06619936880525207

681 17
old [0.58028045] new 0.22690931392663735

683 31
old [0.59016743] new 0.4909283448340941

684 594
old [0.18663202] new 0.962706343948352

685 323
old [0.79452987] new 0.02119095599329323

686 495
old [0.80627005] new 0.0397175441795066

687 700
old [0.1846895] new 0.9890169642914726

688 92
old [0.6883

854 1035
old [0.099773] new 0.9999973340859575

855 2900
old [0.06980656] new 0.9999999999999181

857 476
old [0.85712862] new 0.006838684561546116

858 105
old [0.26453892] new 0.6787277347569577

859 354
old [0.19964153] new 0.9496466800879371

860 71
old [0.68349012] new 0.17069146029480062

861 164
old [0.27515897] new 0.7479891683528089

862 19
old [0.55575463] new 0.36170787002157334

863 359
old [0.22192129] new 0.9204607398621745

864 491
old [0.16685207] new 0.9855421859971653

865 904
old [0.14371397] new 0.9994918160010771

866 112
old [0.29607665] new 0.6622801762903121

867 73
old [0.3136089] new 0.7715135286900646

868 43
old [0.40917873] new 0.7077878166872666

869 494
old [0.21020325] new 0.9591959029507444

870 97
old [0.34986396] new 0.6825880100081232



In [49]:
appro_ks= np.array(appro_ks)
new_predictions=np.array(new_predictions)
np.save("appro_ks_IP.npy", appro_ks)
np.save("new_predictions.npy", new_predictions)
np.save("old_predictions.npy", pred)

# Difference of change and prediction when k increasing

In [ ]:
"""
row = len(list(range(0, X["dev"].shape[0])))
col = len(list(range(1, np.sum(y["train"]))))
changes = np.zeros((row, col))
prediction = np.zeros((row, col))

import warnings
warnings.filterwarnings("ignore")
for i in range(0, X["dev"].shape[0]):
    print(i)
    for j in range(1, np.sum(y["train"]):
        changes[i, j-1], _, prediction[i, j-1] = new_train(j, i, delta_pred)
        
   
np.save("pred_change_app.npy", prediction)
np.save("changes.npy", changes)

difference = prediction - changes
np.save("difference_k_appk.npy", difference)
"""